In [ ]:
import json
import pandas
import time
import os
import csv
from functools import reduce

In [ ]:
inputDir = '..//scrapedData//'
outputDir = '..//summarizedData//'

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourceFiles.json', "r").read())
ncaaData = json.loads(open(inputDir + sourceFiles['ncaa'][0], "r", encoding="utf-8").read())

In [ ]:
## Clean up dirty names
def mungeID(playerString):
    return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state")  

In [ ]:
def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [ ]:
def searchID(identifier, dataList):
    return [element for element in dataList if (element['ID'] == identifier)]

def search(name, team, dataList):
    return [element for element in dataList if (element['name'] == name and element['team'] == team)]

In [ ]:
createNewID(idConfig['ncaa'], ncaaData, '_')

In [ ]:
finalOutput = []
for x in ncaaData:
    if (len(searchID(x['ID'], finalOutput)) == 0):
        playerList = search(x['name'], x['team'], ncaaData)
        finalPlayer = {}
        finalPlayer['ID'] = x['ID']
        finalPlayer['name'] = x['name']
        finalPlayer['team'] = x['team']
        gamesPlayed = 0
        gamesStarted = 0
        year = 2021
        for y in playerList:
            gamesPlayed = gamesPlayed + int(y['gamesPlayed'])
            gamesStarted = gamesStarted + int(y['gamesStarted'])
            if (int(y['year']) < int(year)):
                year = y['year']
        finalPlayer['gamesPlayed'] = gamesPlayed
        finalPlayer['gamesStarted'] = gamesStarted
        finalPlayer['year'] = year
        finalOutput.append(finalPlayer)

In [ ]:
##Delete the columns I no longer care about after creating the ID
##This list will need to expand after setting up blockers and truly linking
for record in finalOutput:
    record['ncaa_gamesPlayed'] = record['gamesPlayed']
    record['ncaa_gamesStarted'] = record['gamesStarted']
    del record['gamesPlayed']
    del record['gamesStarted']
    del record['year']
    del record['name']
    del record['team']
    if ('position' in record.keys()):
        del record['position']
print(finalOutput)

[{'ID': 'thomasallen_indiana', 'ncaa_gamesPlayed': 24, 'ncaa_gamesStarted': 0}, {'ID': 'dominicaltimari_indiana', 'ncaa_gamesPlayed': 0, 'ncaa_gamesStarted': 0}, {'ID': 'marcelinoball_indiana', 'ncaa_gamesPlayed': 41, 'ncaa_gamesStarted': 38}, {'ID': 'ryanbarnes_indiana', 'ncaa_gamesPlayed': 9, 'ncaa_gamesStarted': 0}, {'ID': 'matthewbedford_indiana', 'ncaa_gamesPlayed': 16, 'ncaa_gamesStarted': 15}, {'ID': 'brittbeery_indiana', 'ncaa_gamesPlayed': 23, 'ncaa_gamesStarted': 0}, {'ID': 'justinberry_indiana', 'ncaa_gamesPlayed': 5, 'ncaa_gamesStarted': 0}, {'ID': 'mattbjorson_indiana', 'ncaa_gamesPlayed': 33, 'ncaa_gamesStarted': 2}, {'ID': 'dkbonhomme_indiana', 'ncaa_gamesPlayed': 19, 'ncaa_gamesStarted': 3}, {'ID': 'jeremyboyd_indiana', 'ncaa_gamesPlayed': 2, 'ncaa_gamesStarted': 0}, {'ID': 'dashaunbrown_indiana', 'ncaa_gamesPlayed': 2, 'ncaa_gamesStarted': 0}, {'ID': 'andrebrownjr_indiana', 'ncaa_gamesPlayed': 45, 'ncaa_gamesStarted': 34}, {'ID': 'khalilbryant_indiana', 'ncaa_gamesPlay

In [ ]:
with open(outputDir + "ncaa.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(finalOutput))